In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import random


In [2]:
np.random.seed(0)
#random.seed(0) if actively using random

def random_complex_vector(n=1, max_norm=1, fixed_norm=True):
    """
    Returns a complex vector of dimension n x 1 (defaults to 1 x 1).
    Complex elements have norm ]0,max_norm] uniformly randomly, or norm max_norm if fixed_norm == True
    """
    ret = np.empty(n,dtype=np.complex_)
    for i in range(n):
        if fixed_norm:
            norm = max_norm*(1-random.random()) #to have ]0,max_norm]
        else:
            norm = max_norm
        ang = 2*np.pi*random.random()
        ret[i] = max_norm*np.exp((0+1j)*ang)
    return ret

def random_complex_vector(n=1, distribution='gaussian', param=1/np.sqrt(2)):
    assert distribution in ['gaussian','uniform','fixed_norm'], \
        f"Parameter distribution can not be {distribution}"
    if distribution=='gaussian':
        return np.random.normal(loc=0,scale=param,size=n) + \
            (0+1j)*np.random.normal(loc=0,scale=param,size=n)
    else:
        ret = np.empty(n,dtype=np.complex_)
        for i in range(n):
            if distribution=='fixed_norm':
                norm = param*(1-random.random()) #to have ]0,param]
            elif distribution=='uniform':
                norm = param
            ang = 2*np.pi*random.random()
            ret[i] = norm*np.exp((0+1j)*ang)
    return ret

In [27]:
def random_complex_vector(length=1, distribution='gaussian', param=1/np.sqrt(2)):
    """
    Returns a complex vector of dimension length x 1
    If distribution=='gaussian', complex elements are as x+iy, with x,y ~N(0,param^2), param is std
    If distribution=='uniform', complex elements have norm ]0,param] uniformly randomly, phase ]0,2pi] uniformly randomly
    If distribution=='fixed_norm', complex elements have norm param, phase ]0,2pi] uniformly randomly
    
    Complex standard normal (gaussian) distribution has variance 1/2 over the real and over the imaginary part (total variance 1)
    """
    assert distribution in ['gaussian','uniform','fixed_norm'], \
        f"Parameter distribution can not be {distribution}"

    if distribution=='gaussian':
        return np.random.normal(loc=0,scale=param,size=length) + \
            (0+1j)*np.random.normal(loc=0,scale=param,size=length)
    elif distribution=='fixed_norm':
        return param*np.exp(2*np.pi*(0+1j)*np.random.random(length))
    elif distribution=='uniform':
        return param*(1 - np.random.random(length))*np.exp(2*np.pi*(0+1j)*np.random.random(length)) #to have norm in ]0,param]

vec = random_complex_vector(100000,'gaussian',1/np.sqrt(2))
plt.hist2d(np.real(vec),np.imag(vec), [200,200])
plt.show

np.var(random_complex_vector(10000))

In [4]:
def define_w_hat(dim):
    """
    Returns a complex vector of dimension d x 1, the "teacher" vector to be found.
    Its components are randomly initialized: its norm is in [0,1[, its phase in [0,2pi[.
    Its complex norm squared is d (which means its numpy.linalg.norm is the root of d)
    """
    #assert isinstance(dim,int) and dim > 0, f"Given variable dim should not be {dim}"
    ret = random_complex_vector(dim,'uniform',1)
    return np.sqrt(dim)*ret/np.linalg.norm(ret)

d = 102
np.linalg.norm(define_w_hat(d))

In [74]:
def define_X(n,k,law='gaussian',param=1/np.sqrt(2)):
    """
    Returns a matrix of dimension n x k, that is the data.
    If law=='gaussian', its rows are complex standard normally distributed, meaning x+iy with x,y~N(0,1/2)
    """
    mat = np.empty((n,k),dtype=np.complex_)
    for i in range(n):
        mat[i] = random_complex_vector(k,law)
    return mat

In [122]:
def define_y(X,w_hat):
    """
    Returns an array of dimension n x 1, built from the data X and the teacher vector w_hat
    It keeps only the modulus of every element, that should be in principle in [0,1]
    (but can be bigger if the dimension of w_hat is finite)
    """
    return np.abs(X@w_hat)/np.sqrt(len(w_hat))

$\nu(h,h_0) = \frac{1}{2}(|h|^2-|h_0|^2)^2$
Could very simply be optimized in the code by not taking the norm of $h_0$ which will be a positive real number, $y^i$, anyways

In [128]:
def cost(h,h_0):
    """
    The cost function "mu", comparing |X^i@w| to the value y^i=|X^i@w_hat| it corresponds to, minimized in that value
    """
    return (np.abs(h)**2-np.abs(h_0)**2)**2/2

$\mathcal{L}(\underline{w}) = \sum_{i=1}^{N} \nu(h^i,\hat{h}^i) \hspace{1cm} h^i = \frac{1}{\sqrt{d}}\underline{X}^i\cdot\underline{w}, \hspace{5mm} \hat{h}^i=\frac{1}{\sqrt{d}}\underline{X}^i\cdot\underline{\hat{w}}$

In [129]:
def loss(w,X,y):
    """
    The loss function to be minimized, y^i can be taken in place of \hat{h}^i
    """
    return np.sum(cost(X@w/np.sqrt(len(w)),y))

$s^i(t) = \left\{ \begin{array}{ll} 1 \text{ with probability b} \\
0 \text{ with probability 1-b} \end{array} \right.$
$b \in ]0,1]$

In [180]:
def isinbatch(b=1):
    """
    Gives 1 with a probability b, or 0 otherwise. Defaults to being always 1
    Function s^i(t)
    """
    return np.random.choice([1,0],p=[b,1-b])